In [10]:
import csv
import codecs
import pprint
import xml.etree.cElementTree as ET



OSM_PATH = "LA.osm"
NODE_TAGS_PATH = "nodes_tags.csv"
NODE_TAGS_FIELDS = ['id', 'lat', 'lon', 'name', 'amenity', 'cuisine']



def shape_element(element):
    """Clean and shape node or way XML element to Python dict"""
    tags = []
    desire_tags = ['amenity','name','cuisine']
    if element.tag == 'node':
        for child in element:
            tag1 = {}
            if child.attrib['k'] in desire_tags:
                tag1['id'] = element.attrib["id"]
                tag1['lat'] = element.attrib['lat']
                tag1['lon'] = element.attrib['lon']
                if child.attrib['k'] == 'name':
                    tag1['name'] = child.attrib["v"]
                    tag1['amenity'] = None
                    tag1['cuisine'] = None
                elif child.attrib['k'] == 'amenity':
                    tag1['name'] = None
                    tag1['amenity'] = child.attrib["v"]
                    tag1['cuisine'] = None
                elif child.attrib['k'] == 'cuisine':
                    tag1['name'] = None
                    tag1['amenity'] = None
                    tag1['cuisine'] = child.attrib["v"]
                tags.append(tag1) 
        return {'node_tags': tags}
    

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag""" 
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file: 
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        node_tags_writer.writeheader()
        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node')):
            el = shape_element(element)
            if el:   
                if element.tag == 'node':
                    node_tags_writer.writerows(el['node_tags'])


if __name__ == '__main__':
    process_map(OSM_PATH, validate=True)